In [7]:
import re

def isVariable(x):
    return len(x) == 1 and x.islower() and x.isalpha()

def getAttributes(string):
    expr = '\([^)]+\)'
    matches = re.findall(expr, string)
    print('attributes: ',matches)
    print()
    return matches

def getPredicates(string):
    expr = '([a-z~]+)\([^&|]+\)'
    print('predicates: ',re.findall(expr, string))
    print()
    return re.findall(expr, string)

In [14]:
class Fact:
    def __init__(self, expression):
        self.expression = expression
        predicate, params = self.splitExpression(expression)
        self.predicate = predicate
        self.params = params
        self.result = any(self.getConstants())
        print('exp: ',self.expression)
        print('pred: ',self.predicate)
        print('params: ',self.params)
        print('res: ',self.result)
        
    def splitExpression(self, expression):
        predicate = getPredicates(expression)[0]
        params = getAttributes(expression)[0].strip('()').split(',')
        print('predicate: ',predicate)
        print()
        print('params: ',params)
        print()
        return [predicate, params]
    
    def getResult(self):
        print('result: ',self.result)
        print()
        return self.result
    
    def getConstants(self):
        return [None if isVariable(c) else c for c in self.params]
    
    def getVariables(self):
        return [v if isVariable(v) else None for v in self.params]
    
    def substitute(self, constants):
        c = constants.copy()
        f = f"{self.predicate}({','.join([constants.pop(0) if isVariable(p) else p for p in self.params])})"
        return Fact(f)

In [10]:
class Implication:
    def __init__(self, expression):
        self.expression = expression
        l = expression.split('=>')
        self.lhs = [Fact(f) for f in l[0].split('&')]
        self.rhs = Fact(l[1])
        
    def evaluate(self, facts):
        print('lhs of =>',self.lhs)
        print('rhs of =>',self.rhs)
        constants = {}
        new_lhs = []
        for fact in facts:
            for val in self.lhs:
                if val.predicate == fact.predicate:
                    for i, v in enumerate(val.getVariables()):
                        if v:
                            constants[v] = fact.getConstants()[i]
                    new_lhs.append(fact)
        predicate, attributes = getPredicates(self.rhs.expression)[0], str(getAttributes(self.rhs.expression)[0])
        for key in constants:
            if constants[key]:
                attributes = attributes.replace(key, constants[key])
        expr = f'{predicate}{attributes}'
        print('expr after implication :',expr)
        return Fact(expr) if len(new_lhs) and all([f.getResult() for f in new_lhs]) else None

In [15]:
class KB:
    def __init__(self):
        self.facts = set()
        self.implications = set()
            
    def tell(self, e):
        if '=>' in e:
            self.implications.add(Implication(e))
        else:
            self.facts.add(Fact(e))
        for i in self.implications:
            res = i.evaluate(self.facts)
            if res:
                self.facts.add(res)

    def query(self, e):
        facts = set([f.expression for f in self.facts])
        i = 1
        print(f'Querying {e}:')
        for f in facts:
            if Fact(f).predicate == Fact(e).predicate:
                print(f'\t{i}. {f}')
                i += 1
    
    def display(self):
        print("All facts: ")
        for i, f in enumerate(set([f.expression for f in self.facts])):
            print(f'\t{i+1}. {f}')

In [12]:
def main():
    kb = KB()
    print("Enter KB: (enter e to exit)")
    while True:
        t = input()
        if(t == 'e'):
            break
        kb.tell(t)
    print("Enter Query:")
    q = input()
    kb.query(q)
    kb.display()

In [8]:
main()

Enter KB: (enter e to exit)
missile(x)=>weapon(x)
missile(M1)
enemy(x,America)=>hostile(x)
american(West)
enemy(Nono,America)
owns(Nono,M1)
missile(x)&owns(Nono,x)=>sells(West,x,Nono)
american(x)&weapon(y)&sells(x,y,z)&hostile(z)=>criminal(x)
e
Enter Query:
criminal(x)
Querying criminal(x):
	1. criminal(West)
All facts: 
	1. missile(M1)
	2. weapon(M1)
	3. american(West)
	4. owns(Nono,M1)
	5. criminal(West)
	6. sells(West,M1,Nono)
	7. hostile(Nono)
	8. enemy(Nono,America)


In [ ]:
main()

Enter KB: (enter e to exit)
missile(x)=>weapon(x)
predicates:  ['missile']

attributes:  ['(x)']

predicate:  missile

params:  ['x']

exp:  missile(x)
pred:  missile
params:  ['x']
res:  False
predicates:  ['weapon']

attributes:  ['(x)']

predicate:  weapon

params:  ['x']

exp:  weapon(x)
pred:  weapon
params:  ['x']
res:  False
lhs of => [<__main__.Fact object at 0x0000024B24DFF3C8>]
rhs of => <__main__.Fact object at 0x0000024B24DFF848>
predicates:  ['weapon']

attributes:  ['(x)']

expr after implication : weapon(x)
missile(M1)
predicates:  ['missile']

attributes:  ['(M1)']

predicate:  missile

params:  ['M1']

exp:  missile(M1)
pred:  missile
params:  ['M1']
res:  True
lhs of => [<__main__.Fact object at 0x0000024B24DFF3C8>]
rhs of => <__main__.Fact object at 0x0000024B24DFF848>
predicates:  ['weapon']

attributes:  ['(x)']

expr after implication : weapon(M1)
result:  True

predicates:  ['weapon']

attributes:  ['(M1)']

predicate:  weapon

params:  ['M1']

exp:  weapon(M1)
p